# Cassandra

En **Cassandra** vamos a crear 2 tablas, una donde almacenamos los accidentes en formato desnormalizado, y otra que nos servirá para almacenar el número de accidientes por tipo.

En la tabla que almacena los accidentes vamos a utilizar como **PARTITION KEY** el número de expediente y como **CLUSTERING KEY** la fecha del accidente que nos permitirá ordenar la información tal y como nos interesa para nuestro caso de uso.

Adiccionalmente tendremos un índice en la tabla de accidentes que nos permitirá localizar la información por Distrito

<br><br> 

<img src="https://i.imgur.com/Oy6ppdC.png" width=500 height=500>

<br><br> 

## Carga de datos en Pandas

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("data/accidentes.xlsx", engine='openpyxl')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 686 entries, 0 to 685
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   num_expediente  686 non-null    object        
 1   fecha           686 non-null    datetime64[ns]
 2   distrito        686 non-null    object        
 3   tipo_accidente  686 non-null    object        
 4   sexo            686 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 26.9+ KB


In [3]:
df.sample(10)

,num_expediente,fecha,distrito,tipo_accidente,sexo
181,2020S007104,2020-05-21,HORTALEZA,Colisión fronto-lateral,Hombre
268,2020S008143,2020-06-11,FUENCARRAL-EL PARDO,Caída,Hombre
43,2020S002132,2020-01-29,TETUÁN,Colisión fronto-lateral,Hombre
129,2020S006534,2020-05-03,CENTRO,Colisión fronto-lateral,Hombre
374,2020S009561,2020-07-06,RETIRO,Caída,Hombre
21,2020S000867,2020-01-14,CARABANCHEL,Atropello a persona,Hombre
126,2020S006402,2020-04-28,MONCLOA-ARAVACA,Caída,Hombre
263,2020S008184,2020-06-10,HORTALEZA,Caída,Mujer
609,2020S013704,2020-10-03,CIUDAD LINEAL,Caída,Hombre
17,2020S000749,2020-01-11,ARGANZUELA,Colisión lateral,Mujer


## Creacción de Estructura en Cassandra

In [4]:
%load_ext cql

In [5]:
%%cql
DROP KEYSPACE accidentes;

"Cannot drop non existing keyspace 'accidentes'."

In [6]:
%%cql
CREATE KEYSPACE accidentes 
WITH replication = {'class':'SimpleStrategy', 'replication_factor': 1};

'No results.'

In [7]:
%cql USE accidentes;

'No results.'

In [8]:
%%cql 
CREATE TABLE acum_tipo_accidente (
    tipo_accidente          text,
    total         counter,
    PRIMARY KEY (tipo_accidente)
)

'No results.'

Crea la tabla accidentes de forma que se muestren los datos estén ordenados por fecha

In [ ]:
%%cql 
CREATE TABLE accidentes (
    num_expediente   text,
    fecha            date,
    distrito         text,
    tipo_accidente   text,
    sexo             text,
    PRIMARY KEY <<COMPLETA EL CODIGO>>
)
WITH CLUSTERING ORDER BY (fecha DESC);

Crea un índice para que puedas hacer búsquedas por distrito

%%cql
CREATE INDEX <<COMPLETA EL CODIGO>>

In [ ]:
%%cql
CREATE INDEX accidentes_distrito ON accidentes(distrito)

'No results.'

## Carga de datos en Cassandra

In [11]:
from cassandra.cluster import Cluster, BatchStatement, ConsistencyLevel
cluster = Cluster()
session = cluster.connect('accidentes')

In [12]:
import dateutil

def insert_accidentes(df):
    
    sql_insert = """
INSERT INTO accidentes (
num_expediente,
fecha,
distrito,
tipo_accidente,
sexo
) VALUES (%s, %s, %s, %s, %s)
"""

    for index in df.index:
        
        data = [
            df["num_expediente"][index],
            (df['fecha'][index]).date(),
            df["distrito"][index],
            df["tipo_accidente"][index],
            df["sexo"][index]
        ]
        
        session.execute(sql_insert, data)             
        session.execute("UPDATE acum_tipo_accidente SET total = total + 1 WHERE tipo_accidente = %s", 
                        [df["tipo_accidente"][index]]
        )

In [13]:
insert_accidentes(df)

## Resolución del caso de uso

### Datos del número de expediente 2020S000044

In [ ]:
%%cql 
<<COMPLETA EL CODIGO>>

Comprueba que te muestra el siguiente resultado:

<img src="https://i.imgur.com/ytUbj8N.png">

### Accidentes que ocurren en el distrito de Moratalaz ordenados por fecha

In [ ]:
%%cql 
<<COMPLETA EL CODIGO>>

Comprueba que te muestra el siguiente resultado:

<img src="https://i.imgur.com/EJnFtrJ.png">

### Número de accidentes por tipo

In [ ]:
%%cql 
<<COMPLETA EL CODIGO>>

Comprueba que te muestra el siguiente resultado:

<img src="https://i.imgur.com/dSHPIbr.png">

### Número de accidentes cuyo tipo es colisión frontal

In [ ]:
%%cql
<<COMPLETA EL CODIGO>>

Comprueba que te muestra el siguiente resultado:

<img src="https://i.imgur.com/1oM6HbU.png">